Exercise 1. 

In [47]:
import pandas as pd
import numpy as np
from math import log, sqrt, exp, erf
from datetime import datetime

def norm_cdf(x):
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))

def bs_price(S, K, T, r, q, sigma, callput):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        if callput.lower().startswith("c"):
            return max(S * exp(-q*T) - K * exp(-r*T), 0.0)
        else:
            return max(K * exp(-r*T) - S * exp(-q*T), 0.0)
    d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    if callput.lower().startswith("c"):
        return S * exp(-q*T) * norm_cdf(d1) - K * exp(-r*T) * norm_cdf(d2)
    else:
        return K * exp(-r*T) * norm_cdf(-d2) - S * exp(-q*T) * norm_cdf(-d1)

In [61]:
path = "OptionsPivotTables.xlsx"
raw = pd.read_excel(path, sheet_name="OptionsData")
# Extract value date from header row
value_date = None
for c in raw.columns:
    if isinstance(c, (pd.Timestamp, datetime)):
        value_date = pd.Timestamp(c).normalize()
        break
# Rebuild proper table
header = raw.iloc[2].tolist()
df = raw.iloc[3:].copy()
df.columns = header
underlying_row = df[df.get("Security Type", "").eq("Future")].head(1)
if not underlying_row.empty:
    S0 = float(underlying_row["Underlying Price"].iloc[0])
    q = float(underlying_row["Div. Yield"].iloc[0])
else:
    S0 = float(df["Underlying Price"].dropna().iloc[0])
    q = float(df["Div. Yield"].dropna().iloc[0])
T = (pd.to_datetime(df["Maturity"]) - value_date).dt.days / 365.25

# Scenarios
price_shifts = [-0.20, -0.15, -0.05, 0.0, 0.05, 0.10, 0.20]    
vol_point_shifts = [-5, -2, 0, 2, 5, 15, 20]                   
vol_shifts = [v / 100.0 for v in vol_point_shifts]
scenario_cols = [f"PnL: Und {int(ps*100):+d}% | Vol {vpp:+d}pp"
                 for ps in price_shifts for vpp in vol_point_shifts]
scenario_df = pd.DataFrame(index=df.index, columns=scenario_cols, dtype=float)

# Extract key columns
positions = pd.to_numeric(df["Position"], errors="coerce").fillna(0.0).values
strikes   = pd.to_numeric(df["Strike"], errors="coerce").values
impvols   = pd.to_numeric(df["Imp. Vol"], errors="coerce").values
rates     = pd.to_numeric(df["Risk-Free Rate"], errors="coerce").fillna(0.0).values
opt_px0   = pd.to_numeric(df["Security Price"], errors="coerce").values
mats      = T.values
types     = df["Security Type"].astype(str).values
cpflag    = df["CallPut"].astype(str).values

# Calculate P&L for each scenario
for ps in price_shifts:
    S_scn = S0 * (1.0 + ps)
    for vpp, vs in zip(vol_point_shifts, vol_shifts):
        col = f"PnL: Und {int(ps*100):+d}% | Vol {vpp:+d}pp"
        pnl = np.full_like(positions, np.nan, dtype=float)
        mask_opt = (types == "Option")
        if mask_opt.any():
            sigma_new = np.maximum(impvols[mask_opt] + vs, 1e-6)
            K = strikes[mask_opt]
            T = np.maximum(mats[mask_opt], 1e-8)
            r = rates[mask_opt]
            cp = cpflag[mask_opt]
            px0 = opt_px0[mask_opt]
            pos = positions[mask_opt]
            px_new = np.array([
                bs_price(S_scn, K[i], T[i], r[i], q, sigma_new[i], cp[i])
                for i in range(K.shape[0])
            ])
            pnl[mask_opt] = pos * (px_new - px0)
        mask_fut = (types == "Future")
        if mask_fut.any():
            fut_px0 = pd.to_numeric(df.loc[mask_fut, "Security Price"], errors="coerce").values
            fut_pos = positions[mask_fut]
            pnl[mask_fut] = fut_pos * (S_scn - fut_px0)
        scenario_df[col] = pnl

# Write back to Excel (starting at column AR) 
with pd.ExcelWriter(path, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    scenario_df.to_excel(writer, sheet_name="OptionsData", startrow=2, startcol=43, index=False)

Exercise 2. 

(a)

In [78]:
import pandas as pd

path = "OptionsPivotTables(updated).xlsx"
df = pd.read_excel(path, sheet_name="OptionsData", header=3)
# Create pivot table: Strike (rows), Maturity (columns), filter by Underlying
pivot_table_spx = pd.pivot_table(
    df[df["Underlying"] == "SPX Index"],
    values=scenario_col,
    index="Strike",
    columns="Maturity",
    aggfunc="sum"
)

(b)

In [99]:
import numpy as np
from math import log, sqrt, exp
from scipy.stats import norm

# Since the given workbook does not really compute the greeks, we do this manually 
def bs_greeks(S, K, T, r, q, sigma, callput):
    if pd.isna(K) or pd.isna(T) or pd.isna(r) or pd.isna(sigma):
        return (np.nan,)*5
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return (np.nan,)*5
    
    d1 = (log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    pdf = norm.pdf(d1)
    
    if str(callput).lower().startswith("c"):
        delta = exp(-q*T) * norm.cdf(d1)
    else:
        delta = -exp(-q*T) * norm.cdf(-d1)
    
    gamma = exp(-q*T) * pdf / (S * sigma * sqrt(T))
    vega1 = 0.01 * S * exp(-q*T) * pdf * sqrt(T)
    volga = vega1 * d1 * d2 / sigma
    vanna = - (d2 / sigma) * S * exp(-q*T) * pdf
    
    return (delta, gamma, vega1, volga, vanna)

S0 = float(df["Underlying Price"].iloc[0])
q  = float(df["Div. Yield"].iloc[0])
value_date = pd.to_datetime("2010-01-01")
T = (pd.to_datetime(df["Maturity"]) - value_date).dt.days / 365.25

greek_results = [
    bs_greeks(S0, row["Strike"], T_i, row["Risk-Free Rate"], q, row["Imp. Vol"], row["CallPut"])
    if row["Security Type"]=="Option" else (np.nan,)*5
    for (_, row), T_i in zip(df.iterrows(), T)
]

df[["Delta","Gamma","Vega1%","Volga","Vanna"]] = pd.DataFrame(greek_results, index=df.index)
print(df.loc[df["Security Type"]=="Option",
             ["Strike","Maturity","Delta","Gamma","Vega1%","Volga","Vanna"]].head(10))

    Strike   Maturity     Delta     Gamma    Vega1%     Volga       Vanna
1    935.0 2010-03-19 -0.216957  0.001091  1.480864  1.159981 -324.148692
2   1001.0 2010-03-19 -0.294830  0.001448  1.738205  0.632473 -257.129811
3   1100.0 2010-03-19 -0.464638  0.001916  1.999385 -0.042610  110.721229
4   1034.0 2010-03-19 -0.344834  0.001632  1.855584  0.313724 -172.826343
5   1232.0 2010-03-19  0.281636  0.001758  1.701222  1.928693  731.221049
6   1100.0 2010-03-19  0.531154  0.001916  1.999385 -0.042610  110.721229
7   1034.0 2010-03-19 -0.344834  0.001632  1.855584  0.313724 -172.826343
8   1034.0 2010-03-19 -0.344834  0.001632  1.855584  0.313724 -172.826343
9   1100.0 2010-03-19  0.531154  0.001916  1.999385 -0.042610  110.721229
10  1232.0 2010-03-19  0.281636  0.001758  1.701222  1.928693  731.221049


In [102]:
# Taylor Approximation
scenario_col = "PnL: Und -20% | Vol +20"
dS = -0.20 * S0       
dVol = 0.20          

df["Taylor_Approx"] = (
    dS * df["Delta"] +
    0.5 * (dS**2) * df["Gamma"] +
    dVol * df["Vega1%"] * 100 + 
    0.5 * (dVol**2) * df["Volga"] +
    dS * dVol * df["Vanna"])

# Compare
df["Scenario_PnL"] = df[scenario_col]
df["Difference"] = df["Scenario_PnL"] - df["Taylor_Approx"]

print(df.loc[df["Security Type"]=="Option",
             ["Strike","Maturity","Scenario_PnL","Taylor_Approx","Difference"]].head(10))

    Strike   Maturity  Scenario_PnL  Taylor_Approx    Difference
1    935.0 2010-03-19  22659.491423   14366.306123   8293.185300
2   1001.0 2010-03-19  -6807.212455   11448.391210 -18255.603665
3   1100.0 2010-03-19   4263.761578   -4683.160743   8946.922321
4   1034.0 2010-03-19  -3700.186730    7756.826354 -11457.013084
5   1232.0 2010-03-19   -437.656553  -32159.077499  31721.420946
6   1100.0 2010-03-19  -1213.062140   -4902.235112   3689.172972
7   1034.0 2010-03-19  -7400.373461    7756.826354 -15157.199815
8   1034.0 2010-03-19  -7400.373461    7756.826354 -15157.199815
9   1100.0 2010-03-19   2426.124280   -4902.235112   7328.359392
10  1232.0 2010-03-19   -525.187863  -32159.077499  31633.889635


(c)

No, because the current way that the OptionsData worksheet is organized isn’t well suited to building a pivot table that treats the underlying shift and the volatility shift as separate dimensions. In a pivot table we need these shifts to exist as separate columns instead of being joined in the same column (which rather represents a scenario).

To make such a pivot table possible, the data would have needed to be stored in “long format”. Instead of 49 extra columns, there would be three extra columns, one column recording the underlying price shift, one column recording the volatility shift, and one column recording the P&L. Then each row of the table would represent the P&L of a single option under a single stress scenario (so the number of rows shall multiply by 49 because there is 49 scenarios for each single option (row)). 

Exercise 3. 

In [18]:
import numpy as np
import pandas as pd
from math import sqrt
from scipy.stats import norm, t
import matplotlib.pyplot as plt

# Parameters
mu = 0.0
annual_vol = 0.2
trading_days = 250
scale_multiplier = 10000  # to make numbers readable

sigma_L = annual_vol / sqrt(trading_days) * scale_multiplier  # daily σ
nu = 5  # degrees of freedom for Student's t

# For the t distribution, scale so that Var(L_t) = sigma_L^2
sigma_t_scale = sigma_L * sqrt((nu - 2) / nu)

# Confidence levels as given
alphas = np.array([0.90, 0.95, 0.975, 0.99, 0.995, 0.999, 0.9999, 0.99999, 0.999999])

# ES functions
def ES_normal(mu, sigma, alpha):
    z = norm.ppf(alpha)
    return mu + sigma * norm.pdf(z) / (1 - alpha)

def ES_student_t(mu, scale, nu, alpha):
    q = t.ppf(alpha, df=nu)
    g = t.pdf(q, df=nu)
    ES_std_t = g * (nu + q**2) / ((nu - 1) * (1 - alpha))
    return mu + scale * ES_std_t

# Compute table
rows = []
for a in alphas:
    # Normal
    var_norm = mu + sigma_L * norm.ppf(a)
    es_norm = ES_normal(mu, sigma_L, a)
    ratio_norm = es_norm / var_norm if var_norm != 0 else np.nan

    # Student-t
    var_t = mu + sigma_t_scale * t.ppf(a, df=nu)
    es_t = ES_student_t(mu, sigma_t_scale, nu, a)
    ratio_t = es_t / var_t if var_t != 0 else np.nan

    rows.append([a, var_norm, es_norm, ratio_norm, var_t, es_t, ratio_t])

result_df = pd.DataFrame(rows, columns=[
    "α (confidence)", "VaR Normal", "ES Normal", "ES/VaR Normal", "VaR t5", "ES t5", "ES/VaR t5"
])

print(result_df)

   α (confidence)  VaR Normal   ES Normal  ES/VaR Normal       VaR t5  \
0        0.900000  162.104875  221.989782       1.369421   144.606514   
1        0.950000  208.059355  260.914825       1.254040   197.433613   
2        0.975000  247.918013  295.711262       1.192778   251.864554   
3        0.990000  294.262316  337.125896       1.145665   329.694461   
4        0.995000  325.819499  365.805779       1.122725   395.067715   
5        0.999000  390.886903  425.906949       1.089591   577.435807   
6        0.999900  470.422510  500.712473       1.064389   948.203976   
7        0.999990  539.470755  566.519890       1.050140  1523.274429   
8        0.999999  601.265900  625.920080       1.041004  2427.055329   

         ES t5  ES/VaR t5  
0   225.571541   1.559899  
1   283.173648   1.434273  
2   345.042702   1.369953  
3   436.247178   1.323186  
4   514.395845   1.302045  
5   736.253644   1.275040  
6  1194.204614   1.259439  
7  1909.698023   1.253680  
8  3037.345314   

Exercise 4 (a)

In [27]:
import pandas as pd
import numpy as np
file_path = "SP500_returns.xlsx"  
df = pd.read_excel(file_path)
returns = df['r500'].dropna().values
pos_val = 1000000  # 1 million
alpha = 0.95
# VaR
var_95 = -np.percentile(returns, (1 - alpha) * 100) * pos_val
# Expected Shortfall
threshold = np.percentile(returns, (1 - alpha) * 100)
es_95 = -returns[returns <= threshold].mean() * pos_val
print(f"95% VaR: {var_95:,.2f}")
print(f"95% ES: {es_95:,.2f}")

95% VaR: 15,137.75
95% ES: 23,473.81


Exercise 4 (b)

In [32]:
import numpy as np
import pandas as pd
from scipy.stats import norm, t

alpha = 0.95 # confidence level
pos_val = 1000000  # 1 million

# Fit Student-t distribution
df_t, loc_t, scale_t = t.fit(returns)  # ν, μ, s
# 95% quantile for fitted t
q_t = t.ppf(alpha, df_t, loc=loc_t, scale=scale_t) * pos_val
# VaR
VaR_t = q_t
# ES 
q_std = t.ppf(alpha, df_t)
g = t.pdf(q_std, df_t)
ES_std = g * (df_t + q_std**2) / ((df_t - 1) * (1 - alpha))
ES_t = loc_t + scale_t * ES_std * pos_val
# Compare
results = pd.DataFrame({
    "VaR 95%": [VaR_t],
    "ES 95%": [ES_t]
})
print("Fitted Student-t parameters:")
print(f"df = {df_t:.2f}, loc = {loc_t:.6f}, scale = {scale_t:.6f}\n")
print(results)


Fitted Student-t parameters:
df = 4.12, loc = 0.000510, scale = 0.007247

        VaR 95%        ES 95%
0  15828.114971  22848.765076


The results we obtain for 95% VaR and 95% ES are slightly higher than the results in part (a). 